In [1]:
import pandas as pd
import numpy as np

import seaborn as sn
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split 
from sklearn.linear_model import LinearRegression
from sklearn import metrics
from matplotlib.pylab import (figure, semilogx, loglog, xlabel, ylabel, legend, 
                           title, subplot, show, grid)

import sklearn.linear_model as lm
from sklearn import model_selection

import matplotlib.pyplot as plt
import numpy as np
from scipy.io import loadmat
import torch
from sklearn import model_selection
#from toolbox_02450 import train_neural_net, draw_neural_net
from scipy import stats

%matplotlib inline

In [2]:
df=pd.read_csv('winequality-white.csv',sep=';')
df_plain=pd.read_csv('winequality-white.csv',sep=';')

In [3]:
predict='sulphates'
remove_outliers=True

In [4]:
shape_orig=df.shape[0]
#removal of the 0-1 percentile and 99-100% of the values furthest away from the mean
#I will do my prediction of 'sulphates' therefore I do not remove its outliers
predict='sulphates'
# predict='density'
remove_lim=0.001

lim_high,lim_low=1-remove_lim,remove_lim

if remove_outliers==True:
    features=[cols for cols in df.columns if cols != predict]
    print(50*'_')
    index_names=[]
    #FOR all FEATURES get the percentile and add the indexes to a list (index_names)
    for feat in features:
        y = df[feat]
        removed_outliers = y.between(y.quantile(0.25)-3*(y.quantile(0.75)-y.quantile(0.25)),y.quantile(0.75)+3*(y.quantile(0.75)-y.quantile(0.25)))
#         print(feat,'\n',removed_outliers.value_counts())
        for index in list(df[~removed_outliers].index):
            if index not in index_names:
                index_names.append(index)
    #when all are added remove them
    df.drop(index_names, inplace=True)
    remove_outliers=False
print('% of data dropped',100*(1-(df.shape[0]/shape_orig)))

__________________________________________________
% of data dropped 4.042466312780723


In [6]:
X=df.drop(columns=predict).values
y=df[predict].values
attributeNames = [name for name in df.drop(columns=predict)]
N, M = X.shape
#folds per level
K=10

#regression defs C1
lambdas = np.power(10.,np.linspace(-5,10,num=10))
lambdas_opt=np.zeros(K)
w_rlr = np.empty((M+1,K))
mu = np.empty((K, M-1))
sigma = np.empty((K, M-1))
Error_train_rlr = np.empty((K,1))
Error_test_rlr = np.empty((K,1))

#ANN defs c1
activation=torch.nn.ReLU()
n_replicates = 1        # number of networks trained in each k-fold
max_iter = 10000
max_iter_ = 1000
H = np.array([i for i in range(4,8)])
H_opt=np.empty(K)
fold_summary=[]
loss_fn = torch.nn.MSELoss() # notice how this is now a mean-squared-error loss
train_error_ann = np.empty((K,1))
test_error_ann = np.empty((K,1))

k=0
CV1 = model_selection.KFold(K, shuffle=True)
for train_index, test_index in CV1.split(X,y):
    X_train = X[train_index]
    y_train = y[train_index]
    X_test = X[test_index]
    y_test = y[test_index]
    
    CV2 = model_selection.KFold(K, shuffle=True)
    M_ = X_train.shape[1]
    
    #regression defs c2
    M_reg=M+1
    w = np.empty((M_reg,K,len(lambdas)))
    train_error_reg = np.empty((K,len(lambdas)))
    test_error_reg = np.empty((K,len(lambdas)))
    
    #ANN defs c2
    train_error_ann_ = np.empty((K,len(H)))
    test_error_ann_ = np.empty((K,len(H)))
    
    best_final_loss = 1e100
    
    f = 0
    y_train = y_train.squeeze()
    
    for train_index_, test_index_ in CV2.split(X_train,y_train):
        print('outerfold:',k,'/',K,'innerfold:',f,'/',K)
        X_train_ = X[train_index_]
        y_train_ = y[train_index_]
        X_test_ = X[test_index_]
        y_test_ = y[test_index_]
        
        #add w0 for reg
        X_train_reg=np.concatenate((np.ones((X_train_.shape[0],1)),X_train_),1)
        X_test_reg=np.concatenate((np.ones((X_test_.shape[0],1)),X_test_),1)
        
        #settings for NN
        logging_frequency = 250 # display the loss every 1000th iteration
        best_final_loss_ = 1e100
        tolerance=1e-6
        
        #standardize
        mu_ = np.mean(X_train_[:, 1:], 0)
        sigma_ = np.std(X_train_[:, 1:], 0)
        X_train_[:, 1:] = (X_train_[:, 1:] - mu_) / sigma_
        X_test_[:, 1:] = (X_test_[:, 1:] - mu_) / sigma_
        
        #regression
        Xty = X_train_reg.T @ y_train_
        XtX = X_train_reg.T @ X_train_reg
        #run regression model to find optimal lambds C2
        for l in range(0,len(lambdas)):
            # Compute parameters for current value of lambda and current CV fold
            # note: "linalg.lstsq(a,b)" is substitue for Matlab's left division operator "\"
            lambdaI = lambdas[l] * np.eye(M_reg)
            lambdaI[0,0] = 0 # remove bias regularization
            w[:,f,l] = np.linalg.solve(XtX+lambdaI,Xty).squeeze()
            # Evaluate training and test performance
            train_error_reg[f,l] = np.power(y_train_-X_train_reg @ w[:,f,l].T,2).mean(axis=0)
            test_error_reg[f,l] = np.power(y_test_-X_test_reg @ w[:,f,l].T,2).mean(axis=0)
        
        #find optimal h ANN C2
        print('\t\t{}\t{}\t\t\t{}'.format('Iter', 'Loss','Rel. loss'))
        for count,h in enumerate(H):
            print('n_hidden_units:',h)
            model_ = lambda: torch.nn.Sequential(
                    torch.nn.Linear(M, h), #M features to n_hidden_units
                    activation,   # 1st transfer function,
                    torch.nn.Linear(h, 1), # n_hidden_units to 1 output neuron
                    )
            net_ = model_()
        
            # initialize weights based on limits that scale with number of in- and
            # outputs to the layer, increasing the chance that we converge to 
            # a good solution
            torch.nn.init.xavier_uniform_(net_[0].weight)
            torch.nn.init.xavier_uniform_(net_[2].weight)
            optimizer = torch.optim.Adam(net_.parameters())
            
            learning_curve_ = [] # setup storage for loss at each step
            old_loss_ = 1e6
            
            for i in range(max_iter_):
                y_est = net_(torch.Tensor(X_train_)) # forward pass, predict labels on training set
                y_est_test = net_(torch.Tensor(X_test_))
                loss = loss_fn(y_est.squeeze(), torch.Tensor(y_train_)) # determine loss
                loss_value_ = loss.data.numpy() #get numpy array instead of tensor
                learning_curve_.append(loss_value_) # record loss for later display

                # Convergence check, see if the percentual loss decrease is within
                # tolerance:
                p_delta_loss = np.abs(loss_value_-old_loss_)/old_loss_
                if p_delta_loss < tolerance: break
                old_loss_ = loss_value_

                # display loss with some frequency:
                if (i != 0) & ((i+1) % logging_frequency == 0):
                    print_str = '\t\t' + str(i+1) + '\t' + str(loss_value_) + '\t' + str(p_delta_loss)
                    print(print_str)
                # do backpropagation of loss and optimize weights 
                optimizer.zero_grad(); loss.backward(); optimizer.step()
            if loss_value_ < best_final_loss_:
                best_net_ = net_
                best_final_loss_ = loss_value_
                best_learning_curve_ = learning_curve_
                train_error_ann_[f,count] = np.power(y_train_-y_est.detach().numpy().squeeze(),2).mean(axis=0)
                test_error_ann_[f,count] = np.power(y_test_-y_est_test.detach().numpy().squeeze(),2).mean(axis=0)
                print('new best:',best_final_loss_)
        
        
        f=f+1
    #return optimal lambda
    opt_val_err = np.min(np.mean(test_error_reg,axis=0))
    opt_lambda = lambdas[np.argmin(np.mean(test_error_reg,axis=0))]
    train_err_vs_lambda = np.mean(train_error_reg,axis=0)
    test_err_vs_lambda = np.mean(test_error_reg,axis=0)
    mean_w_vs_lambda = np.squeeze(np.mean(w,axis=1))
    lambdas_opt[k]=opt_lambda
    #return optimal h
    best_h=H[np.argmin(np.mean(test_error_ann_,axis=0))]
    H_opt[k]=best_h
    
    #baseline
    baseline=np.square(y_test-y_test.mean()).sum(axis=0)/y_test.shape[0]
    
    #regression C1
    mu[k, :] = np.mean(X_train[:, 1:], 0)
    sigma[k, :] = np.std(X_train[:, 1:], 0)
    X_train[:, 1:] = (X_train[:, 1:] - mu[k, :] ) / sigma[k, :] 
    X_test[:, 1:] = (X_test[:, 1:] - mu[k, :] ) / sigma[k, :]
    X_train_reg=np.concatenate((np.ones((X_train.shape[0],1)),X_train),1)
    X_test_reg=np.concatenate((np.ones((X_test.shape[0],1)),X_test),1)
 
    Xty = X_train_reg.T @ y_train
    XtX = X_train_reg.T @ X_train_reg
    
    lambdaI = opt_lambda * np.eye(M+1)
    lambdaI[0,0] = 0 # Do no regularize the bias term
    w_rlr[:,k] = np.linalg.solve(XtX+lambdaI,Xty).squeeze()
    # Compute mean squared error with regularization with optimal lambda
    Error_train_rlr[k] = np.square(y_train-X_train_reg @ w_rlr[:,k]).sum(axis=0)/y_train.shape[0]
    Error_test_rlr[k] = np.square(y_test-X_test_reg @ w_rlr[:,k]).sum(axis=0)/y_test.shape[0]
    
    #ANN C1
    model_ = lambda: torch.nn.Sequential(
        torch.nn.Linear(M, best_h), #M features to n_hidden_units
        activation,   # 1st transfer function,
        torch.nn.Linear(best_h, 1), # n_hidden_units to 1 output neuron
        )
    net = model_()

    # initialize weights based on limits that scale with number of in- and
    # outputs to the layer, increasing the chance that we converge to 
    # a good solution
    torch.nn.init.xavier_uniform_(net[0].weight)
    torch.nn.init.xavier_uniform_(net[2].weight)
    optimizer = torch.optim.Adam(net.parameters())

    learning_curve = [] # setup storage for loss at each step
    old_loss = 1e6
    
    for i in range(max_iter):
        y_est = net(torch.Tensor(X_train)) # forward pass, predict labels on training set
        y_est_test = net(torch.Tensor(X_test))
        loss = loss_fn(y_est.squeeze(), torch.Tensor(y_train)) # determine loss
        loss_value = loss.data.numpy() #get numpy array instead of tensor
        learning_curve.append(loss_value) # record loss for later display

        # Convergence check, see if the percentual loss decrease is within
        # tolerance:
        p_delta_loss = np.abs(loss_value-old_loss)/old_loss
        if p_delta_loss < tolerance: break
        old_loss = loss_value

        # display loss with some frequency:
        if (i != 0) & ((i+1) % logging_frequency == 0):
            print_str = '\t\t' + str(i+1) + '\t' + str(loss_value) + '\t' + str(p_delta_loss)
            print(print_str)
        # do backpropagation of loss and optimize weights 
        optimizer.zero_grad(); loss.backward(); optimizer.step()
    if loss_value < best_final_loss:
        best_net = net
        best_final_loss = loss_value
        best_learning_curve = learning_curve
        train_error_ann[k] = np.power(y_train-y_est.detach().numpy().squeeze(),2).mean(axis=0)
        test_error_ann[k] = np.power(y_test-y_est_test.detach().numpy().squeeze(),2).mean(axis=0)
        print('new best:',best_final_loss)
    
    summary_tags=['fold','h_opt','E_test_ann','lambda_opt','E_test_lrl','baseline']
    fold_summary.append([k+1,best_h,np.round(test_error_ann[k][0],4),opt_lambda,np.round(Error_test_rlr[k][0],4),np.round(baseline,4)])
    k+=1

summary_tags=['fold','h_opt','E_test_ann','lambda_opt','E_test_lrl','baseline']
print(summary_tags[0],'\t',summary_tags[1],'\t',summary_tags[2],'\t',summary_tags[3],'\t',summary_tags[4],'\t',summary_tags[5])
for i in fold_summary:
    print(i[0],'\t',i[1],'\t',i[2],'\t\t',i[3],'\t\t',i[4],'\t\t',i[5])    
#run both models with optimal lambda,h

outerfold: 0 / 10 innerfold: 0 / 10
		Iter	Loss			Rel. loss
n_hidden_units: 4
		250	0.026332727	0.0041782563
		500	0.015880762	0.00084435666
		750	0.014120877	0.00024726483
		1000	0.013570354	0.00010025721
new best: 0.013570354
n_hidden_units: 5
		250	0.07326042	0.005473269
		500	0.027787473	0.0026931716
		750	0.015976205	0.0014364052
		1000	0.0132811	0.00037901622
new best: 0.0132811
n_hidden_units: 6
		250	0.03412985	0.002641215
		500	0.021815205	0.0013761832
		750	0.016375268	0.00090767635
		1000	0.013468597	0.0006231854
n_hidden_units: 7
		250	0.11899051	0.005266025
		500	0.044892304	0.0029763393
		750	0.024391439	0.0020150905
		1000	0.016275864	0.0012360478
outerfold: 0 / 10 innerfold: 1 / 10
		Iter	Loss			Rel. loss
n_hidden_units: 4
		250	0.027077716	0.0067624045
		500	0.015542749	0.0006157786
		750	0.014292528	0.00017727302
		1000	0.013884222	7.465198e-05
new best: 0.013884222
n_hidden_units: 5
		250	0.4965822	0.011287763
		500	0.08516116	0.0038617977
		750	0.045139726	0.0018657

		250	0.019712057	0.0030311805
		500	0.014187455	0.00048448113
		750	0.013264802	0.00014433105
		1000	0.012951935	6.2554354e-05
new best: 0.012951935
n_hidden_units: 5
		250	0.013570937	0.00049345364
		500	0.012874997	9.757141e-05
		750	0.012648648	5.647118e-05
		1000	0.01247824	5.5973665e-05
new best: 0.01247824
n_hidden_units: 6
		250	0.0689131	0.008391199
		500	0.028071051	0.0020474193
		750	0.018923005	0.001193843
		1000	0.015042342	0.0006918367
n_hidden_units: 7
		250	0.020643285	0.0036647492
		500	0.013898224	0.00063525484
		750	0.012728861	0.00018244325
		1000	0.012335334	9.436659e-05
new best: 0.012335334
outerfold: 1 / 10 innerfold: 1 / 10
		Iter	Loss			Rel. loss
n_hidden_units: 4
		250	0.013944469	0.00061847444
		500	0.0131288655	0.00012440805
		750	0.012753124	0.00011244895
		1000	0.012478394	5.522671e-05
new best: 0.012478394
n_hidden_units: 5
		250	0.013909874	0.0008532704
		500	0.012456665	0.00023806984
		750	0.012054219	6.883493e-05
		1000	0.011869735	6.2922525e-05
new b

		250	0.07767529	0.0053585498
		500	0.02941379	0.0027440093
		750	0.018211951	0.0012354996
		1000	0.014836261	0.00051278394
n_hidden_units: 6
		250	0.38297936	0.008869663
		500	0.09737189	0.0025569096
		750	0.058549907	0.001818107
		1000	0.037632473	0.0016639445
n_hidden_units: 7
		250	0.018490857	0.0039627966
		500	0.0134846335	0.00043519924
		750	0.012568069	0.0001849251
		1000	0.012131548	0.00011183942
new best: 0.012131548
outerfold: 2 / 10 innerfold: 1 / 10
		Iter	Loss			Rel. loss
n_hidden_units: 4
		250	0.1111539	0.0069210147
		500	0.03391391	0.0022268707
		750	0.023686	0.001073371
		1000	0.018854683	0.00077095133
new best: 0.018854683
n_hidden_units: 5
		250	0.04910706	0.007160275
		500	0.017576762	0.0015467032
		750	0.0143579105	0.0004164529
		1000	0.013413254	0.00017528761
new best: 0.013413254
n_hidden_units: 6
		250	0.022301532	0.007052283
		500	0.014073593	0.00037394842
		750	0.013283177	0.00014497254
		1000	0.012935517	7.883085e-05
new best: 0.012935517
n_hidden_units: 7
	

		250	0.043282554	0.0030446057
		500	0.026355594	0.0013831458
		750	0.020119347	0.00084333826
		1000	0.016902404	0.000577445
n_hidden_units: 6
		250	0.031274643	0.006495771
		500	0.01595851	0.0009492991
		750	0.014046296	0.0002581203
		1000	0.013480756	0.00010320284
n_hidden_units: 7
		250	0.03308917	0.0033534956
		500	0.018932065	0.0012931802
		750	0.015312996	0.0005635361
		1000	0.0137590645	0.00032756993
outerfold: 3 / 10 innerfold: 1 / 10
		Iter	Loss			Rel. loss
n_hidden_units: 4
		250	0.12064244	0.010204899
		500	0.023586735	0.0028370605
		750	0.015832864	0.00088002725
		1000	0.013823279	0.00031850775
new best: 0.013823279
n_hidden_units: 5
		250	0.015501899	0.0015521046
		500	0.0130204195	0.00031240753
		750	0.012335583	0.00016010746
		1000	0.011888288	0.00014678668
new best: 0.011888288
n_hidden_units: 6
		250	0.088188194	0.008071495
		500	0.033522066	0.0026224286
		750	0.019970827	0.0014989963
		1000	0.015598163	0.00056218525
n_hidden_units: 7
		250	0.042252336	0.006861241
		50

		250	0.05711567	0.009971208
		500	0.018447721	0.0019234854
		750	0.013675669	0.0007197826
		1000	0.012176101	0.00028131926
new best: 0.012176101
n_hidden_units: 5
		250	0.03468812	0.0065432605
		500	0.014801618	0.0011941842
		750	0.012995124	0.00021108665
		1000	0.012545724	0.00010065152
n_hidden_units: 6
		250	0.045084484	0.007753401
		500	0.016578723	0.0014807322
		750	0.013604185	0.00041838037
		1000	0.012640377	0.00021421086
n_hidden_units: 7
		250	0.017931588	0.0022507778
		500	0.013889853	0.00045888458
		750	0.012934574	0.00017723891
		1000	0.012524259	9.487626e-05
outerfold: 4 / 10 innerfold: 2 / 10
		Iter	Loss			Rel. loss
n_hidden_units: 4
		250	0.099117815	0.009185799
		500	0.017408082	0.002995209
		750	0.013261845	0.00027254192
		1000	0.012836823	6.268002e-05
new best: 0.012836823
n_hidden_units: 5
		250	0.13312282	0.0036663858
		500	0.047277346	0.0036444825
		750	0.024227504	0.0018447434
		1000	0.01733806	0.0009638006
n_hidden_units: 6
		250	0.039025698	0.0045212316
		500	0

		500	0.018845318	0.0017395922
		750	0.01455235	0.00054554193
		1000	0.013350361	0.00022429817
outerfold: 5 / 10 innerfold: 2 / 10
		Iter	Loss			Rel. loss
n_hidden_units: 4
		250	0.08539176	0.009460911
		500	0.02238734	0.0020942625
		750	0.016267464	0.0008208722
		1000	0.0140712615	0.00038961836
new best: 0.0140712615
n_hidden_units: 5
		250	0.027905282	0.006851888
		500	0.014497578	0.0007530946
		750	0.013023071	0.00025981176
		1000	0.012419005	0.00013991498
new best: 0.012419005
n_hidden_units: 6
		250	0.03905303	0.0034156758
		500	0.022352617	0.0014028077
		750	0.017029889	0.0008859181
		1000	0.014136901	0.00060381
n_hidden_units: 7
		250	0.053299878	0.005242862
		500	0.02030456	0.0023481261
		750	0.014642871	0.00066077424
		1000	0.013201005	0.00025976583
outerfold: 5 / 10 innerfold: 3 / 10
		Iter	Loss			Rel. loss
n_hidden_units: 4
		250	0.02474637	0.0036077728
		500	0.013615769	0.0012614182
		750	0.011726484	0.0002932931
		1000	0.011204964	0.0001064615
new best: 0.011204964
n_hidde

		1000	0.012496329	0.00062779954
outerfold: 6 / 10 innerfold: 2 / 10
		Iter	Loss			Rel. loss
n_hidden_units: 4
		250	0.07322299	0.011176582
		500	0.022164611	0.0014375665
		750	0.017611198	0.00063693285
		1000	0.015522713	0.0004023007
new best: 0.015522713
n_hidden_units: 5
		250	0.2066557	0.010542871
		500	0.0710037	0.0017165737
		750	0.047102805	0.001572886
		1000	0.033390272	0.0011382598
n_hidden_units: 6
		250	0.050519288	0.0042383047
		500	0.023800615	0.002112627
		750	0.016472306	0.0009702792
		1000	0.01386805	0.00048087415
new best: 0.01386805
n_hidden_units: 7
		250	0.02147787	0.004275
		500	0.014182038	0.0005434452
		750	0.013109549	0.00018360867
		1000	0.012690179	9.36359e-05
new best: 0.012690179
outerfold: 6 / 10 innerfold: 3 / 10
		Iter	Loss			Rel. loss
n_hidden_units: 4
		250	0.03471902	0.0050171823
		500	0.016939208	0.0011930766
		750	0.014301291	0.00039343993
		1000	0.013295364	0.0002236154
new best: 0.013295364
n_hidden_units: 5
		250	0.10325637	0.0046575316
		500	0.05

		250	0.08667479	0.007329856
		500	0.029621184	0.002952318
		750	0.018029373	0.0012205824
		1000	0.014795378	0.0004826323
new best: 0.014795378
n_hidden_units: 5
		250	0.040862657	0.00548178
		500	0.017824182	0.0015507899
		750	0.014371918	0.0004727629
		1000	0.013245912	0.00022416892
new best: 0.013245912
n_hidden_units: 6
		250	0.02737259	0.0033768634
		500	0.016183287	0.001190409
		750	0.013587125	0.00040822139
		1000	0.012610583	0.000219589
new best: 0.012610583
n_hidden_units: 7
		250	0.014285726	0.00060292793
		500	0.013417563	0.00010563194
		750	0.013178285	5.1657833e-05
		1000	0.013041484	3.649036e-05
outerfold: 7 / 10 innerfold: 3 / 10
		Iter	Loss			Rel. loss
n_hidden_units: 4
		250	0.025340594	0.004165637
		500	0.015556156	0.00084887166
		750	0.013824149	0.0002419318
		1000	0.013310508	9.2910166e-05
new best: 0.013310508
n_hidden_units: 5
		250	0.015638666	0.0018784447
		500	0.012989873	0.00026413015
		750	0.01242412	0.00014053185
		1000	0.0119845765	0.00015601753
new best: 0

		250	0.024176091	0.004481942
		500	0.014106613	0.0010397276
		750	0.011991773	0.00036954146
		1000	0.011316081	0.0001391513
new best: 0.011316081
n_hidden_units: 5
		250	0.039387096	0.006897217
		500	0.015772557	0.0014505677
		750	0.012951319	0.0004156078
		1000	0.012054363	0.00019882823
n_hidden_units: 6
		250	0.028115094	0.0026258242
		500	0.018271767	0.001353575
		750	0.013954076	0.00072515925
		1000	0.012462616	0.00027111912
n_hidden_units: 7
		250	0.025074549	0.0043808795
		500	0.015086607	0.0008084022
		750	0.013469257	0.00024187744
		1000	0.012943818	0.00010590087
outerfold: 8 / 10 innerfold: 4 / 10
		Iter	Loss			Rel. loss
n_hidden_units: 4
		250	0.049444675	0.0075146565
		500	0.018227924	0.0016102104
		750	0.014428832	0.0005606543
		1000	0.0129893385	0.00028743036
new best: 0.0129893385
n_hidden_units: 5
		250	0.04662644	0.0038268038
		500	0.025091646	0.0017190385
		750	0.018122347	0.0009593679
		1000	0.015057071	0.0005627913
n_hidden_units: 6
		250	0.027612135	0.003857727
		5

		250	0.037965894	0.005155791
		500	0.01615086	0.0016222192
		750	0.012906502	0.00048171898
		1000	0.011955819	0.00019953233
new best: 0.011955819
n_hidden_units: 6
		250	0.117006786	0.011084163
		500	0.04508522	0.0012180175
		750	0.032429557	0.0014123226
		1000	0.023219038	0.001217308
n_hidden_units: 7
		250	0.019496381	0.0027828037
		500	0.013674745	0.0005819614
		750	0.012669477	0.00015037712
		1000	0.012354853	7.077786e-05
outerfold: 9 / 10 innerfold: 4 / 10
		Iter	Loss			Rel. loss
n_hidden_units: 4
		250	0.08750006	0.01010749
		500	0.018000087	0.002062848
		750	0.01419738	0.00045655173
		1000	0.013135841	0.00021357411
new best: 0.013135841
n_hidden_units: 5
		250	0.057212308	0.005786358
		500	0.017336221	0.0027554824
		750	0.012799922	0.00047976754
		1000	0.011859471	0.0001966001
new best: 0.011859471
n_hidden_units: 6
		250	0.05122857	0.0076946504
		500	0.01984195	0.0015712327
		750	0.015531646	0.00063765835
		1000	0.0138021	0.00035210527
n_hidden_units: 7
		250	0.019424979	0.003

In [ ]:
params=dict(net.named_parameters())
params

# beta = 0.5 #The interpolation parameter    
# params1 = model1.named_parameters()
# params2 = model2.named_parameters()

# dict_params2 = dict(params2)

# for name1, param1 in params1:
#     if name1 in dict_params2:
#         dict_params2[name1].data.copy_(beta*param1.data + (1-beta)*dict_params2[name1].data)

# model.load_state_dict(dict_params2)

In [ ]:
from sklearn import model_selection
import scipy.stats as st
test_proportion = 0.2

X_train, X_test, y_train, y_test = model_selection.train_test_split(X,y,test_size=test_proportion)

#rlr
mu_= np.mean(X_train[:, 1:], 0)
sigma_= np.std(X_train[:, 1:], 0)
X_train[:, 1:] = (X_train[:, 1:] - mu_ ) / sigma_ 
X_test[:, 1:] = (X_test[:, 1:] - mu_ ) / sigma_

X_train_reg=np.concatenate((np.ones((X_train.shape[0],1)),X_train),1)
X_test_reg=np.concatenate((np.ones((X_test.shape[0],1)),X_test),1)

Xty = X_train_reg.T @ y_train
XtX = X_train_reg.T @ X_train_reg

lambdaI = opt_lambda * np.eye(M+1)
lambdaI[0,0] = 0 # Do no regularize the bias term
w_rlr_ = np.linalg.solve(XtX+lambdaI,Xty).squeeze()
# Compute mean squared error with regularization with optimal lambda
# Error_train_rlr_ = np.square(y_train-X_train_reg @ w_rlr_).sum(axis=0)/y_train.shape[0]
# Error_test_rlr_ = np.square(y_test-X_test_reg @ w_rlr_).sum(axis=0)/y_test.shape[0]
Zrlr=np.square(y_test-X_test_reg @ w_rlr_)

#NN
y_est_test = net(torch.Tensor(X_test))
Zann=np.power(y_test-y_est_test.detach().numpy().squeeze(),2)

#baseline
Zbase=np.square(y_test-y_test.mean())

alpha = 0.05

CI=np.empty([3,2])
p=np.zeros(3)
Z = (Zbase-Zrlr,Zbase-Zann,Zrlr-Zann)

for c,z in enumerate(Z):
    CI[c] = st.t.interval(1-alpha, len(z)-1, loc=np.mean(z), scale=st.sem(z))  # Confidence interval
    p[c] = st.t.cdf( -np.abs( np.mean(z) )/st.sem(z), df=len(z)-1)  # p-value

In [ ]:
CI,p,[min(i) for i in Z],[max(i) for i in Z]

In [ ]:
[sum(i)/len(i) for i in Z]

In [ ]:
(Zbase).mean()-Zann.mean(),Zrlr.mean()

In [ ]:
Zbase

In [ ]:
st.sem(z),np.std(z)/np.sqrt(len(z))

In [ ]:
CI=np.empty([3,2])
p=np.zeros(3)
Z = (Zrlr-Zbase,Zann-Zbase,Zann-Zrlr)

for c,z in enumerate(Z):
    CI[c] = st.t.interval(1-alpha, len(z)-1, loc=np.mean(z), scale=st.sem(z))  # Confidence interval
    p[c] = st.t.cdf( -np.abs( np.mean(z) )/st.sem(z), df=len(z)-1)  # p-value

In [ ]:
(y_test,X_test_reg @ w_rlr_)
(y_test,y_est_test.detach().numpy())
plt.scatter(y_test,[y_test.mean() for i in range(len(y_test))])
plt.xlim(0,1)
plt.ylim(0,1)

In [ ]:
plt.scatter(y_test,y_est_test.detach().numpy())
plt.xlim(0,1)
plt.ylim(0,1)

In [ ]:
plt.scatter(y_test,X_test_reg @ w_rlr_)
plt.xlim(0,1)
plt.ylim(0,1)